In [2]:
import websocket
import asyncio
import os
import json
from datetime import datetime
import base64
import hmac
import time
import requests

In [4]:
type(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

str

In [1]:
from bitget.ws.client import BitgetWsClient

In [5]:
client = BitgetWsClient('wss://ws.bitget.com/mix/v1/stream')
client.build()

start connecting... url:  wss://ws.bitget.com/mix/v1/stream
start connecting... url:  wss://ws.bitget.com/mix/v1/stream
connection is success....


Keep connected:pong
Keep connected:pong
Keep connected:pong


In [2]:
BITGET_ACCESS_KEY = os.environ.get("BITGET_ACCESS_KEY")
BITGET_SECRET_KEY = os.environ.get("BITGET_SECRET_KEY")
BITGET_PASSPHRASE = os.environ.get("BITGET_PASSPHRASE")

In [25]:
def start_time():
    return time.time()

def time_elapsed(start):
    return time.time()-start

def get_sign(message, secret_key):
    mac = hmac.new(bytes(secret_key, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
    d = mac.digest()
    return base64.b64encode(d)


def pre_hash(timestamp, method, request_path, body):
    return str(timestamp) + str.upper(method) + request_path + body

def get_timestamp(self):
    url = "https://api.bitget.com/api/spot/v1/public/time"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['data']
    else:
        return ""

def on_open(ws):
    timestamp = get_timestamp()
    sign = get_sign(pre_hash(timestamp, 'GET', '/user/verify', ''), BITGET_SECRET_KEY)
    msg = json.dumps({
        "op":"login",
        "args":[
            {
                "apiKey": BITGET_ACCESS_KEY,
                "passphrase": BITGET_PASSPHRASE,
                "timestamp": str(timestamp),
                "sign": bytes.decode(sign)
            }
        ]
    })
    ws.send(msg)
    print("Opened connection")

def on_message(ws, msg):
    print(msg)

def on_close(ws, close_status_code, close_msg):
    print("### Closed connection ###")

In [33]:
now = datetime.now()
# epoch time
epoch_time = datetime(1970, 1, 1)
# subtract datetime from Unix epoch datetime
delta = (now - epoch_time).total_seconds()
# timestamp = get_timestamp()
sign = get_sign(pre_hash(timestamp, 'GET', '/user/verify', ''), BITGET_SECRET_KEY)
msg = json.dumps({
    "op":"login",
    "args":[
        {
            "apiKey": BITGET_ACCESS_KEY,
            "passphrase": BITGET_PASSPHRASE,
            "timestamp": str(timestamp),
            "sign": bytes.decode(sign)
        }
    ]
})
int(delta*1000), get_timestamp(), int(time.time() * 1000)

(1665692032073, 1665692032073, 1665706432073)

In [27]:
now

datetime.datetime(2022, 10, 13, 20, 1, 41, 264301)

In [26]:
socket = "wss://ws.bitget.com/mix/v1/stream"    
websocket.enableTrace(True)             
# ws = websocket.WebSocketApp(socket, 
#                             on_open=on_open, 
#                             on_message=on_message,
#                             on_close=on_close) 

# wst = threading.Thread(target=lambda: ws.run_forever())
# wst.daemon = True
# wst.start()
ws = websocket.WebSocket()
ws.connect(socket)
on_open(ws)
print(ws.recv())
ws.close()

--- request header ---
GET /mix/v1/stream HTTP/1.1
Upgrade: websocket
Host: ws.bitget.com
Origin: https://ws.bitget.com
Sec-WebSocket-Key: Q76f2xDYNKiYqJW+cpXDvw==
Sec-WebSocket-Version: 13
Connection: Upgrade
Cookie: __cf_bm=LUa5q7IFkMYlzry6QRtt2ieFIqarOPuMd_At8LCundU-1665704918-0-Ae0jY1xVyvGKs6N0uIsoDj6BtdZyMl6JQeHgMCubRLmr4SOzEgUqT8NgPNrEj0GqT8HSuzCwa57zJaiD0cIIntw=


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Fri, 14 Oct 2022 00:02:06 GMT
Connection: upgrade
upgrade: websocket
sec-websocket-accept: JgHgSULT9BRVITEdX1TjgviCtzU=
CF-Cache-Status: DYNAMIC
Server: cloudflare
CF-RAY: 759c0b544c0a2286-MIA
-----------------------
++Sent raw: b'\x81\xfe\x00\xc4?\x7f:\xb7D]U\xc7\x1dE\x1a\x95S\x10]\xdeQ]\x16\x97\x1d\x1eH\xd0L]\x00\x97d\x04\x18\xd6O\x16q\xd2F]\x00\x97\x1d\x1d]\xe8\x0eF\x0f\xd4^\x1d\x0e\xd3\x0bFX\x81\x0b\x1bX\xd4\x06N\\\x80\r\x1a\x03\x8e[\x1b\x08\xd3\x08N\t\xd3\x1dS\x1a\x95O\x1eI\xc4O\x17H\xd6L\x1a\x18\x8d\x1f]y\xc5F\x0fN\xd8k\r[\xd3Z

Opened connection


++Rcv raw: b'\x81@{"event":"error","code":30014,"msg":"Timestamp request expired"}'
++Rcv decoded: fin=1 opcode=1 data=b'{"event":"error","code":30014,"msg":"Timestamp request expired"}'
++Sent raw: b'\x88\x82\xb9dg\xab\xba\x8c'
++Sent decoded: fin=1 opcode=8 data=b'\x03\xe8'


{"event":"error","code":30014,"msg":"Timestamp request expired"}


In [ ]:
from 